In [1]:
import os

In [2]:
%pwd

'E:\\RajaRajeshwari\\MyFolders\\Projects\\mlops_water_potability_prediction\\notebooks'

In [3]:
os.chdir("../")

In [4]:
%pwd

'E:\\RajaRajeshwari\\MyFolders\\Projects\\mlops_water_potability_prediction'

#### Entity

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    source_url: str
    root_dir: Path
    unzip_dir: Path
    zip_data_path: Path

#### Configuration

In [6]:
from src.mlops_water_potability_prediction_project.constants import *
from src.mlops_water_potability_prediction_project.utilities.helpers import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories(directories_path_list=[self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            source_url=config.source_url,
            root_dir=config.root_dir,
            unzip_dir=config.unzip_dir,
            zip_data_path=config.zip_data_path
        )

        return data_ingestion_config

#### Component

In [7]:
import os
import urllib.request as request
import zipfile
from src.mlops_water_potability_prediction_project import logger
from src.mlops_water_potability_prediction_project.utilities.helpers import get_file_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.zip_data_path):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename=self.config.zip_data_path
            )
            logger.info(f"{filename} download! \nWith the following info: \n{headers}")
        else:
            logger.info(f"File already exists. Size : {get_file_size(Path(self.config.zip_data_path))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.zip_data_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

#### Pipeline

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-01-28 17:17:12,449]: INFO: helpers: YAML file: config\config.yaml loaded successfully]
[2024-01-28 17:17:12,451]: INFO: helpers: YAML file: params.yaml loaded successfully]
[2024-01-28 17:17:12,452]: INFO: helpers: YAML file: schema.yaml loaded successfully]
[2024-01-28 17:17:12,453]: INFO: helpers: Created directory at: artifacts]
[2024-01-28 17:17:12,454]: INFO: helpers: Created directory at: artifacts/data_ingestion]
[2024-01-28 17:17:38,820]: INFO: 1011695674: artifacts/data_ingestion/water_potability_dataset.zip download! 
With the following info: 
Connection: close
Content-Length: 249968
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "cdaebd4188ea87522630b86d68bf39fdb01d971a5225484728d80a7634b4d99b"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 3E4C:0E61:1699023:1799C11:65B67E